<a href="https://colab.research.google.com/github/ritwikraha/computer-needs-glasses/blob/master/image-generation/Insanley-Fast-Whisper-SDXL_Lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qq diffusers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 29.1 MB/s eta 0:00:00


In [2]:
!pip install -q pipx && apt install python3.10-venv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 5.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-pip-whl python3-setuptools-whl
The following NEW packages will be installed:
  python3-pip-whl python3-setuptools-whl python3.10-venv
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 2,473 kB of archives.
After this operation, 2,884 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-pip-whl all 22.0.2+dfsg-1ubuntu0.4 [1,680 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-setuptools-whl all 59.6.0-1.2ubuntu0.22.04.1 [788 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3.10-venv amd64 3.10.12-1~22.04.3 [5,716 B]
Fetched 

In [3]:
import torch
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, EulerDiscreteScheduler
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [4]:
base = "stabilityai/stable-diffusion-xl-base-1.0"
repo = "ByteDance/SDXL-Lightning"
ckpt = "sdxl_lightning_2step_unet.safetensors" # Use the correct ckpt for your step

In [5]:
unet = UNet2DConditionModel.from_config(base, subfolder="unet").to("cuda", torch.float16)
unet.load_state_dict(load_file(hf_hub_download(repo, ckpt), device="cuda"))
pipe = StableDiffusionXLPipeline.from_pretrained(base, unet=unet, torch_dtype=torch.float16, variant="fp16").to("cuda")

/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:244: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a model, please use <class 'diffusers.models.unets.unet_2d_condition.UNet2DConditionModel'>.load_config(...) followed by <class 'diffusers.models.unets.unet_2d_condition.UNet2DConditionModel'>.from_config(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)


unet/config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

sdxl_lightning_2step_unet.safetensors:   0%|          | 0.00/5.14G [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

...

vae/config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

tokenizer_2/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
# Ensure sampler uses "trailing" timesteps.
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")

In [7]:
!pipx run insanely-fast-whisper --file-name /content/ttz-e01.wav

/root/.cache/pipx/fbbfdc728a6642e/lib/python3.10/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
config.json: 100% 1.27k/1.27k [00:00<00:00, 5.89MB/s]
model.safetensors: 100% 3.09G/3.09G [00:48<00:00, 64.1MB/s]
generation_config.json: 100% 3.90k/3.90k [00:00<00:00, 15.3MB/s]
tokenizer_config.json: 100% 283k/283k [00:00<00:00, 708kB/s]
vocab.json: 100% 1.04M/1.04M [00:00<00:00, 1.29MB/s]
tokenizer.json: 100% 2.48M/2.48M [00:00<00:00, 2.48MB/s]
merges.txt: 100% 494k/494k [00:00<00:00, 827kB/s]
normalizer.json: 100% 52.7k/52.7k [00:00<00:00, 265kB/s]
added_tokens.json: 100% 34.6k/34.6k [00:00<00:00, 81.6MB/s]
special_tokens_map.json: 100% 2.07k/2.07k [00:00<00:00, 11.2MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
preproc

In [8]:
!head output.json

{"speakers": [], "chunks": [{"timestamp": [0.0, 13.72], "text": " Hello and welcome to Tends to Zero, the podcast that makes math accessible and interesting for everyone."}, {"timestamp": [14.34, 18.6], "text": " Today we are diving into the fascinating world of probability."}, {"timestamp": [19.38, 26.34], "text": " Have you ever wondered how we predict the odds of, say, your favorite sports team winning their next game"}, {"timestamp": [26.34, 32.46], "text": " or how likely it is to roll a six on a die then buckle up for this journey."}, {"timestamp": [33.82, 42.0], "text": " Let's start with the sample space. Imagine you're playing a playlist on shuffle. The sample space"}, {"timestamp": [42.0, 47.22], "text": " is like your whole playlist. It contains all the songs you"}, {"timestamp": [47.22, 54.86], "text": " could possibly listen to each time you hit the next button. In probability it's"}, {"timestamp": [54.86, 60.9], "text": " similar if we are rolling a die for instance our s

In [9]:
import json

with open("output.json") as f:
  data = json.load(f)

text = ""
for chunk in data["chunks"]:
  text += chunk["text"] + " "

print(text.strip())

Hello and welcome to Tends to Zero, the podcast that makes math accessible and interesting for everyone.  Today we are diving into the fascinating world of probability.  Have you ever wondered how we predict the odds of, say, your favorite sports team winning their next game  or how likely it is to roll a six on a die then buckle up for this journey.  Let's start with the sample space. Imagine you're playing a playlist on shuffle. The sample space  is like your whole playlist. It contains all the songs you  could possibly listen to each time you hit the next button. In probability it's  similar if we are rolling a die for instance our sample space includes all  possible outcomes that is every number from 1 to 6.  Next, we have the event space. This is like a specific genre in your playlist.  Suppose you're really in the mood for rock music. Each time you hit next, hoping for a rock song you're in event space. In our die rolling  example an event might be rolling a number greater than 4

In [10]:
# Ensure using the same inference steps as the loaded model and CFG set to 0.
pipe(text, num_inference_steps=2, guidance_scale=0).images[0].save("output.png")

Token indices sequence length is longer than the specified maximum sequence length for this model (462 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ["sample space. imagine you're playing a playlist on shuffle. the sample space is like your whole playlist. it contains all the songs you could possibly listen to each time you hit the next button. in probability it's similar if we are rolling a die for instance our sample space includes all possible outcomes that is every number from 1 to 6. next, we have the event space. this is like a specific genre in your playlist. suppose you're really in the mood for rock music. each time you hit next, hoping for a rock song you're in event space. in our die rolling example an event might be rolling a number greater than 4. and now we arrive at probability. this is where we calculate how likely an event is to happen.

  0%|          | 0/2 [00:00<?, ?it/s]